# Benson Sandbox

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
%matplotlib inline

## Read and prepare data

Read the data into a `pandas` DataFrame.

In [2]:
url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_180106.txt'

In [3]:
df = pd.read_csv(url)

Rename the columns to be more Python-friendly and convert date and time information to the `pandas` datetime format.

In [4]:
df.columns = [c.lower().strip() for c in df.columns]
df.rename(columns={'c/a': 'ca'}, inplace=True)
dt_temp = df.date + ' ' + df.time
df['datetime'] = pd.to_datetime(dt_temp, format='%m/%d/%Y %H:%M:%S')

Drop unnecessary columns.

In [5]:
df.drop(['desc', 'division', 'date', 'time'], axis=1, inplace=True)

Calculate the change in the number of entries from each turnstile reading.  First, group the rows and apply a shift to `entries` and `exits`.  Then subtract the previous reading from the current reading.  Finally, drop null values, unnecessary columns, and drop unreasonably high/low entry and exit volumes.

In [9]:
df['entries_n'] = df.groupby(['station', 'linename', 'ca', 'unit', 'scp']).entries.shift()
df['exits_n'] = df.groupby(['station', 'linename', 'ca', 'unit', 'scp']).exits.shift()

In [ ]:
df['entries_d'] = df.entries - df.entries_n
df['exits_d'] = df.exits - df.exits_n

In [ ]:
df.drop(['entries', 'entries_n', 'exits', 'exits_n'], axis=1, inplace=True)
df.rename(columns={'entries_d': 'entries', 'exits_d': 'exits'}, inplace=True)
df.dropna(inplace=True)
df = df[(df.entries >= 0) & (df.entries < 2e6)]
df = df[(df.exits >= 0) & (df.exits < 2e6)]
df.reset_index(inplace=True)

## Calculate sums

Aggregate sums of entries and exits for all stations.

In [ ]:
sums = df.groupby(['station', 'linename'])[['entries', 'exits']].sum()

These are the top 20 stations by entries.

In [ ]:
top_20_entries = sums.sort_values('entries', ascending=False).head(20)
top_20_entries

These are the top 20 stations by exits.

In [ ]:
top_20_exits = sums.sort_values('exits', ascending=False).head(20)
top_20_exits

## Search along a particular line

One-hot-encode line names.  There are some mistakes here because ancilary lines like SIR and PATH are also encoded as line 1.  Enter line name as query to filter sums to only stations along the requested line.

In [ ]:
one_hot = df.linename.apply(lambda x: list(x)).str.join('|').str.get_dummies()
df = df.join(one_hot)

In [ ]:
search_line = 'L'

df[df[search_line] == 1].groupby(['station', 'linename'])\
                            [['entries', 'exits']]\
                            .sum().sort_values('entries', ascending=False)

## Time-of-day Analysis

### Entries

Pivot the DataFrame to show average entries at each station at each hour's reading.  Resulting DataFrame has many null values where readings were not captured in a particuar hour.

In [ ]:
a = df.groupby(['station', 'linename', df.datetime.dt.hour])[['entries', 'exits']].mean().reset_index()
a['station_line'] = a.station + "_" + a.linename
hourly = a.pivot('datetime', 'station_line', 'entries')
hourly.head()

In [ ]:
targets = list(top_20_entries.reset_index().station + "_" + top_20_entries.reset_index().linename)

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(hourly[targets], cmap='BuGn');

### Exits

In [ ]:
hourly = a.pivot('datetime', 'station_line', 'exits')
hourly

In [ ]:
targets = list(top_20_exits.reset_index().station + "_" + top_20_exits.reset_index().linename)

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(hourly[targets], cmap='BuGn');

To take this forward, should backwards-fill the null values.  Not sure how to do this, need to ask.